Студент Жилкин Федор, 344
# Домашняя работа 6: "Решение уравнения теплопроводности методом сеток."
**Задания:**
1. Явная разностная схема
2. Схема с весами

$$\frac{d^2u}{dt^2}=cos(x)\frac{d^2u}{dx^2}+f(x,t),$$
$$u(x,0)=\phi(x), \frac{du}{dt}\bigg|_{t=0} = \psi(x), 0\leq x \leq 1,$$
$$u(0,t)=\alpha(t), \frac{du}{dx}\bigg|_{x=1} = \beta(t), 0\leq t \leq 1.$$

In [123]:
import numpy as np
import pandas as pd 
import sympy
import matplotlib.pyplot as plt 

**Зададим сетку**

In [124]:
# Разбиваем отрезок [0,1] на N равных частей
N = 5
h = 1/N
x = np.arange(0, 1 + h, h)

T = 1
# Разобьем отрезок [0, T] на M равных частей
M = 5
tau = T/M
t = np.arange(0, T + tau, tau)

**Разберемся с функциями**

$a(x,t)=cos(x) + 0*t = cos(x)$

$b(x,t) = 0$

$c(x,t) = 0$

In [125]:
def a(x):
    return np.cos(x)

Пусть $u(x,t) = x^3 + t^3,$ тогда $u(x,0)=\phi(x)=x^3$

In [126]:
def phi(x):
    return x**3

Т.к. $u(x,t) = x^3 + t^3,$ тогда $u(0,t)=\alpha(t)=t^3$

In [127]:
def alpha(t):
    return t**3

$\frac{du}{dx}(1, t) = 3 = \beta(t)$

In [128]:
def beta(t):
    return 3

Пусть $f(x, t)=xt^2$

In [129]:
def f(x, t):
    return x*t**2

Заменяем L разностным оператором

In [130]:
def L(i, k, u):
    return a(x[i])*((u[k][i+1] - 2*u[k][i] + u[k][i-1])/(h**2))

## Явная разностная схема

In [171]:
def explicit_difference_scheme():
    
    """
    Решение явной разностной схемой.
    Возвращаем решения при k = 0...M
    """

    # Помещать решения будем в матрицу u
    u = []

    # Находим u^0_i
    u_0i = phi(x)
    u.append(u_0i)

    # Находим решения u^k_i при k = 1...M 
    for k in range(1, M+1):
        u_k = [u[k-1][i] + tau*(L(i, k-1, u) + f(x[i], t[k-1])) for i in range(1, N)]

        u_0k = alpha(t[k])
        u_k.insert(0, u_0k)

        u_Nk = 2/3*h*beta(t[k]) + 4*u_k[N-1] + 2*u_k[N-2]
        u_k.insert(N, u_Nk)

        u_copy = np.copy(u_k)
        u.append(u_copy)
        u_k.clear()

    return u

## Схема с весами

In [240]:
def weights_sheme():
    
    """
    Решение схемой с весами.
    Возвращаем решения при k = 0...M
    """

    # Помещать решения будем в матрицу u
    u = []

    # Находим u^0_i
    u_0i = phi(x)
    u.append(u_0i)
    
    for k in range (1, M+1):
        
        # Реализуем трехдиагональную матрицу m
        m = np.zeros((N+1, N+1))

        # Первыя ряд
        m[0][0] = 1
        m[0][1] = 0

        # Последний ряд
        m[N][N-1] = -1/h
        m[N][N] = -1/h

        # Столбец справа
        G = np.zeros(N+1)
        G[0] = alpha(t[k])
        G[N] = beta(t[k])
        
        # Заполняем матрицу
        for i in range(1,N):
            m[i][i-1] = a(x[i])/h**2
            m[i][i] = -2*a(x[i])/h**2
            m[i][i+1] = a(x[i])/h**2
            
            G[i] = -1/tau*u[k-1][i] - (1-1/2)*L(i, k-1, u) - f(x[i], t[k])
            
        u.append(np.linalg.solve(m, G))
    
    return u

In [241]:
pd.DataFrame(weights_sheme())

,0,1,2,3,4,5
0,0.000,0.008000,0.064000,0.216000,0.512000,1.000000
1,0.008,0.063556,0.093153,0.060158,-0.098343,-0.501657
2,0.064,-0.050914,-0.167125,-0.275046,-0.339445,-0.260555
3,0.216,0.068147,-0.071607,-0.185470,-0.264911,-0.335089
4,0.512,0.346605,0.158030,-0.039060,-0.227026,-0.372974
5,1.000,0.798063,0.528822,0.212152,-0.128693,-0.471307


In [242]:
pd.DataFrame(explicit_difference_scheme())

,0,1,2,3,4,5
0,0.000,0.008000,0.064000,0.216000,0.512000,1.000000
1,0.008,0.243216,0.506109,0.810242,1.180838,6.743837
2,0.064,0.380444,0.699228,1.089319,19.275144,79.679215
3,0.216,0.398309,1.040421,74.545785,166.369419,814.969246
4,0.512,2.665894,336.626722,150.182586,2105.975565,8724.667433
5,1.000,1628.655924,-2059.945640,8990.582316,18349.442671,91379.335315
